In [1]:
from collections import Counter, defaultdict, deque
import datetime
import itertools
import logging
import os
import sys

import edlib
import networkx as nx
import numpy as np

sys.path.append('../')


In [2223]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from assembly_utils.map_utils import map_monoreads_to_monoassembly
from sequence_graph.db_graph import DeBruijnGraph
from sequence_graph.idb_graph import get_idb_monostring_set
from sequence_graph.db_graph_3col import DeBruijnGraph3Color
from sequence_graph.path_graph import PathDeBruijnGraph, IDBMappings
from sd_parser.sd_parser import SD_Report

from utils.various import filter_sublsts_n2_dict, index


In [1228]:
k = 300
cen = 'X'

In [1144]:
now = datetime.datetime.now()
date = f'{now.year}{now.month:02}{now.day:02}'
date

'20200928'

In [6]:
## cen6
# monomers_fn = '/Poppy/abzikadze/centroFlye/centroFlye_repo/data/monomers/cen6/cen6_monomers_w_hybrids_chm13.fasta'
# sd_report_assembly_fn = '/Poppy/abzikadze/centroFlye/centroFlye_repo/experiments/20200808/cF_cen6_mid1/SD_assembly/final_decomposition.tsv'
# assembly_fn = '/Poppy/abzikadze/centroFlye/centroFlye_repo/data-share/assemblies/human/chm13/cen6/cFM6_v0.3.0_polish_combined.fasta'
# sd_report_assembly = SD_Report(sd_report_fn=sd_report_assembly_fn, monomers_fn=monomers_fn, sequences_fn=assembly_fn, mode='assembly')


In [7]:
## cenX
monomers_fn = '/Poppy/abzikadze/centroFlye/centroFlye_repo/data/monomers/cenX/cenX_monomers_w_hybrids_chm13_20200617.fasta'
sd_report_assembly_fn = '/Poppy/abzikadze/centroFlye/centroFlye_repo/experiments/20200831/cF_cenX_mid1/SD_assembly/final_decomposition.tsv'
assembly_fn = '/Poppy/abzikadze/centroFlye/centroFlye_repo/data/assemblies/cenX/cFX_v0.8.3_tQ_57ef8d6.fasta'
sd_report_assembly = SD_Report(sd_report_fn=sd_report_assembly_fn, monomers_fn=monomers_fn, sequences_fn=assembly_fn, mode='assembly')


In [8]:
monoassembly = sd_report_assembly.monostring_set.monostrings
raw_monoassembly = next(iter(monoassembly.values())).raw_monostring
len(raw_monoassembly)
# raw_monoassembly

18133

In [9]:
# dbs_assembly, _ = get_idb_monostring_set(string_set=sd_report_assembly.monostring_set,
#                                          mink=k, maxk=k,
#                                          outdir=f'../../experiments/{date}/idb_cen{cen}_assembly_k{k}',
#                                          mode='assembly')

In [10]:
## cen6
# dbs_assembly = {}
# dbs_assembly[k] = \
#    DeBruijnGraph.from_pickle('/Poppy/abzikadze/centroFlye/centroFlye_repo/experiments/20200831/cF_cen6_mid1/idb/db_k300.pickle')



In [1229]:
## cenX
dbs_assembly = {}
dbs_assembly[k] = \
   DeBruijnGraph.from_pickle(f'/Poppy/abzikadze/centroFlye/centroFlye_repo/experiments/20200831/cF_cenX_mid1/idb/db_k{k}.pickle')



In [12]:
## cen6
# reads_fn = '/Poppy/abzikadze/centroFlye/centroFlye_repo/data/reads/cen6/centromeric_reads_6_rel5__S1C6H1L.d6z1_rel5_rds.fasta'

# sd_report_fn = '/Poppy/abzikadze/centroFlye/centroFlye_repo/experiments/20200609/SD_a967a0a_cen6_chm13_rel5_with_hybrids/final_decomposition.tsv'

# sd_report = SD_Report(sd_report_fn=sd_report_fn, monomers_fn=monomers_fn, sequences_fn=reads_fn, mode='ont')



In [13]:
## cenX
reads_fn = '/Poppy/abzikadze/centroFlye/centroFlye_repo/data/reads/cenX/centromeric_reads_X_rel5.fasta'

sd_report_fn = '/Poppy/abzikadze/centroFlye/centroFlye_repo/experiments/20200617/SD_8327dec_cenX_rel5/final_decomposition.tsv'

sd_report = SD_Report(sd_report_fn=sd_report_fn, monomers_fn=monomers_fn, sequences_fn=reads_fn, mode='ont')



In [14]:
monoreads_set = sd_report.monostring_set

In [15]:
locations = map_monoreads_to_monoassembly(monoreads_set, monoassembly)

# New mappings

In [1912]:
from itertools import count, chain

In [2328]:
from utils.various import filter_sublsts_n2_dict

In [2329]:
x = list(range(10))

In [2334]:
class IDBMappings:
    def __init__(self, mappings):
        mappings = filter_sublsts_n2_dict(mappings)
        self.mappings = mappings
    
    def add(self, st, en, new):
        for r_id, mapping in self.mappings.items():
            additions = [0]
            for i, a, b in zip(count(), mapping, mapping[1:]):
                if a == st and b == en:
                    additions.append(i+1)
            new_mapping = []
            for p1, p2 in zip(additions, additions[1:]):
                new_mapping.append(mapping[p1:p2])
                new_mapping.append([new])
            new_mapping.append(mapping[additions[-1]:])
            new_mapping = list(itertools.chain.from_iterable(new_mapping))
            self.mappings[r_id] = new_mapping
    
    def remove(self, edge):
        for r_id, mapping in self.mappings.items():
            self.mappings[r_id] = list(filter(lambda e: e != edge, mapping))
    
    def merge(self, st, en):
        # merge en into st
        for r_id, mapping in self.mappings.items():
            if len(mapping) and mapping[0] == en:
                self.mappings[r_id][0] = st
        self.remove(en)
        
    def get_active_connections(self):
        ac = set()
        for mapping in self.mappings.values():
            for a, b in zip(mapping, mapping[1:]):
                ac.add((a, b))
        return ac

# Light DB

In [2335]:
class LightMappedDBGraph:
    def __init__(self, nx_graph,
                 edge2seq, edge2index, index2edge,
                 max_edge_index, max_node_index,
                 k,
                 idb_mappings):
        self.nx_graph = nx_graph
        self.edge2seq = edge2seq
        self.edge2index = edge2index
        self.index2edge = index2edge
        self.max_edge_index = max_edge_index
        self.max_node_index = max_node_index
        self.k = k
        self.idb_mappings = idb_mappings
        
        self.assert_validity()

    @classmethod
    def fromDB(cls, db, string_set, neutral_symbs=None, raw_mappings=None):
        if raw_mappings is None:
            raw_mappings = db.map_strings(string_set,
                only_unique_paths=True,
                neutral_symbs=neutral_symbs)

        nx_graph = nx.MultiDiGraph()
        edge2seq = {}
        edge_index = 0
        edge2index = {}
        index2edge = {}
        for u, v, key, data in db.nx_graph.edges(data=True, keys=True):
            nx_graph.add_edge(u, v, key)
            edge2index[(u, v, key)] = edge_index
            index2edge[edge_index] = (u, v, key)
            edge2seq[edge_index] = list(data['string'])
            edge_index += 1
            
        mappings = {}
        for r_id, (raw_mapping, _, _) in raw_mappings.items():
            mappings[r_id] = [edge2index[edge] for edge in raw_mapping]
        
        idb_mappings = IDBMappings(mappings)
        max_node_index = 1 + max(nx_graph.nodes)

        return cls(nx_graph=nx_graph,
                   edge2seq=edge2seq,
                   edge2index=edge2index,
                   index2edge=index2edge,
                   max_edge_index=edge_index,
                   max_node_index=max_node_index,
                   k=db.k,
                   idb_mappings=idb_mappings)
        
    def move_edge(self, e1_st, e1_en, e1_key,
                  e2_st, e2_en, e2_key=None):
        old_edge = (e1_st, e1_en, e1_key)
        i = self.edge2index[old_edge]
        self.nx_graph.remove_edge(*old_edge)
        e2_key = self.nx_graph.add_edge(e2_st, e2_en, key=e2_key)
        new_edge = (e2_st, e2_en, e2_key)
        self.edge2index[new_edge] = i
        del self.edge2index[old_edge]
        self.index2edge[i] = new_edge
    
    def remove_edge(self, edge=None, index=None, moving=True):
        assert (edge is None) != (index is None)
        if edge is None:
            edge = self.index2edge[index]
        else:
            index = self.edge2index[edge]
        self.idb_mappings.remove(index)
        self.nx_graph.remove_edge(*edge)
        del self.edge2index[edge]
        del self.index2edge[index]
        del self.edge2seq[index]
        
        if moving:
            for e in list(self.nx_graph.in_edges(edge[1], keys=True)):
                self.move_edge(*e, e[0], edge[0])

            for e in list(self.nx_graph.out_edges(edge[1], keys=True)):
                self.move_edge(*e, edge[0], e[1])

    
    def get_new_vertex_index(self):
        self.max_node_index += 1
        return self.max_node_index - 1
    
    def merge_edges(self, e1, e2):
        # merge edge e2 into e1
        assert self.nx_graph.degree(e1[1]) == 1
        assert self.nx_graph.degree(e2[0]) == 1
        i = self.edge2index[e1]
        j = self.edge2index[e2]
        print('merge idb', i, j, e1, e2)
        self.idb_mappings.merge(i, j)
        in_seq = self.edge2seq[i]
        out_seq = self.edge2seq[j]
        assert in_seq[-self.k+1:] == out_seq[:self.k-1]
        seq = in_seq + out_seq[self.k-1:]
        print(len(in_seq), len(out_seq[self.k-1:]))
        self.edge2seq[i] = seq
        self.move_edge(*e1, e1[0], e2[1])
        self.remove_edge(edge=e2, moving=False)
        
    def add_edge(self, i, j, seq):
        in_edge = self.index2edge[i]
        out_edge = self.index2edge[j]
        new_edge = (in_edge[1], out_edge[0])
        key = self.nx_graph.add_edge(*new_edge)
        new_edge = (*new_edge, key)
        self.edge2index[new_edge] = self.max_edge_index
        self.index2edge[self.max_edge_index] = new_edge
        self.edge2seq[self.max_edge_index] = seq
        self.idb_mappings.add(i, j, self.max_edge_index)
        self.max_edge_index += 1
        
    def __process_vertex(self, u):
        def process_simple():
            if indegree == 1 and outdegree == 1:
                # node on nonbranching path - should not be happening
                assert False

            if indegree == 0 and outdegree == 0:
                # isolate - should be removed
                self.nx_graph.remove_node(u)

            elif indegree == 0 and outdegree > 0:
                # starting vertex
                for j in out_indexes[1:]:
                    old_edge = self.index2edge[j]
                    new_edge = (self.get_new_vertex_index(), old_edge[1], 0)
                    self.move_edge(*old_edge, *new_edge)

            elif indegree > 0 and outdegree == 0:
                # ending vertex
                for i in in_indexes[1:]:
                    old_edge = self.index2edge[i]
                    new_edge = (old_edge[0], self.get_new_vertex_index(), 0)
                    self.move_edge(*old_edge, *new_edge)

            elif indegree == 1 and outdegree > 1:
                # simple 1-in vertex
                assert len(in_indexes) == 1
                in_index = in_indexes[0]
                in_seq = self.edge2seq[in_index]
                c = in_seq[-self.k]
                for j in out_indexes:
                    assert self.edge2seq[j][:self.k-1] == in_seq[-self.k+1:]
                    self.edge2seq[j].insert(0, c)

            elif indegree > 1 and outdegree == 1:
                # simple 1-out vertex
                assert len(out_indexes) == 1
                out_index = out_indexes[0]
                out_seq = self.edge2seq[out_index]
                c = out_seq[self.k-1]
                for i in in_indexes:
                    assert self.edge2seq[i][-self.k+1:] == out_seq[:self.k-1]
                    self.edge2seq[i].append(c)

        def process_complex():
            # complex vertex
            for i in in_indexes:
                old_edge = self.index2edge[i]
                new_edge = (old_edge[0], self.get_new_vertex_index(), 0)
                self.move_edge(*old_edge, *new_edge)

            for j in out_indexes:
                old_edge = self.index2edge[j]
                new_edge = (self.get_new_vertex_index(), old_edge[1], 0)
                self.move_edge(*old_edge, *new_edge)
            
            all_ac = self.idb_mappings.get_active_connections()
            ac_s2e = defaultdict(set)
            ac_e2s = defaultdict(set)
            for e_in in in_indexes:
                for e_out in out_indexes:
                    if (e_in, e_out) in all_ac:
                        ac_s2e[e_in].add(e_out)
                        ac_e2s[e_out].add(e_in)

            merged = {}
            for i in ac_s2e:
                for j in ac_s2e[i]:
                    if i in merged:
                        print(f'{i} -> {merged[i]}')
                        i = merged[i]
                    if j in merged:
                        print(f'{i} -> {merged[j]}')
                        j = merged[j]
                    e_i = self.index2edge[i]
                    e_j = self.index2edge[j]
                    in_seq = self.edge2seq[i]
                    out_seq = self.edge2seq[j]
                    assert in_seq[-self.k+1:] == out_seq[:self.k-1]
                    if len(ac_s2e[i]) == len(ac_e2s[j]) == 1:
                        self.merge_edges(e_i, e_j)
                        merged[j] = i
                    elif len(ac_s2e[i]) >= 2 and len(ac_e2s[j]) >= 2:
                        seq = in_seq[-self.k:] + [out_seq[self.k-1]]
                        assert len(seq) == self.k + 1
                        self.add_edge(i, j, seq)
                    elif len(ac_s2e[i]) == 1 and len(ac_e2s[j]) >= 2:
                        # extend left edge to the right
                        self.move_edge(*e_i, e_i[0], e_j[0])
                        seq = in_seq + [out_seq[self.k-1]]
                        self.edge2seq[i] = seq
                    elif len(ac_e2s[j]) == 1 and len(ac_s2e[i]) >= 2:
                        # extend right edge to the left
                        self.move_edge(*e_j, e_i[1], e_j[1])
                        seq = [in_seq[-self.k]] + out_seq
                        self.edge2seq[j] = seq
                    else:
                        print(len(ac[i]), len(ac[j]))
                        assert False
            
            assert self.nx_graph.in_degree(u) == 0
            assert self.nx_graph.out_degree(u) == 0
            self.nx_graph.remove_node(u)
        
        in_indexes = [self.edge2index[e_in]
                      for e_in in self.nx_graph.in_edges(u, keys=True)]
        out_indexes = [self.edge2index[e_out]
                       for e_out in self.nx_graph.out_edges(u, keys=True)]

        indegree = self.nx_graph.in_degree(u)
        outdegree = self.nx_graph.out_degree(u)

        if indegree >= 2 and outdegree >= 2:
            process_complex()
        else:
            process_simple()

    def assert_validity(self):
        self.max_edge_index == 1 + max(self.index2edge)
        self.max_node_index == 1 + max(self.nx_graph.nodes)
        edges = set(self.nx_graph.edges(keys=True))
        assert edges == set(self.edge2index.keys())
        assert edges == set(self.index2edge.values())
        assert all(edge == self.index2edge[self.edge2index[edge]]
                   for edge in self.edge2index)
        
        ac = self.idb_mappings.get_active_connections()
        for i, j in ac:
            _, u, _ = self.index2edge[i]
            v, _, _ = self.index2edge[j]
            if u != v:
                print(i, j, self.index2edge[i], self.index2edge[j])
            assert u == v

        for node in self.nx_graph.nodes:
            assert self.nx_graph.in_degree(node) != 1 or \
                self.nx_graph.out_degree(node) != 1
            for in_edge in self.nx_graph.in_edges(node, keys=True):
                e_index = self.edge2index[in_edge]
                in_seq = self.edge2seq[e_index]
                for out_edge in self.nx_graph.out_edges(node, keys=True):
                    e_outdex = self.edge2index[out_edge]
                    out_seq = self.edge2seq[e_outdex]
                    if in_seq[-self.k+1:] != out_seq[:self.k-1]:
                        print(e_index, e_outdex, in_edge, out_edge)
                    assert in_seq[-self.k+1:] == out_seq[:self.k-1]
        
    def increase_k_by_one(self):           
        for u in list(self.nx_graph.nodes):
            self.__process_vertex(u)

        self.k += 1
        collapsed_edges = [self.edge2index[edge] for edge in self.nx_graph.edges
                           if len(self.edge2seq[self.edge2index[edge]]) == self.k - 1]
        print([(edge, self.index2edge[edge]) for edge in collapsed_edges])
        # if self.k != 335:
        [self.remove_edge(index=index) for index in collapsed_edges]  # remove collapsed edges
        # else:
        #     print('not removing edges')
            
        self.assert_validity()
        
    def toDB(self):
        nx_graph = nx.MultiDiGraph()
        nodeindex2label = {}
        nodelabel2index = {}
        for i, (u, v, key) in self.index2edge.items():
            seq = tuple(self.edge2seq[i])
            u_label = seq[:self.k-1]
            v_label = seq[-self.k+1:]
            nodelabel2index[u_label] = u
            nodelabel2index[v_label] = v
            nodeindex2label[u] = u_label
            nodeindex2label[v] = v_label
            edge_len = len(seq) - self.k + 1
            cov = [1] * edge_len
            mean_cov = np.mean(cov)
            label = f'index={i}\nlen={edge_len}\ncov={mean_cov:0.2f}'
            nx_graph.add_edge(u, v, key=key,
                              coverage=cov,
                              edge_index=i,
                              edge_len=edge_len,
                              label=label,
                              string=seq,
                              color='black')
        db = DeBruijnGraph(k=self.k,
                           nx_graph=nx_graph,
                           nodeindex2label=nodeindex2label,
                           nodelabel2index=nodelabel2index)
        return db

In [2336]:
# graph with a loop

class DBLoop3:
    k=3
    nx_graph = nx.MultiDiGraph()
    nx_graph.add_edge(0, 1, string='CCAA')
    nx_graph.add_edge(1, 1, string='AACAA')
    nx_graph.add_edge(1, 1, string='AAGAA')
    nx_graph.add_edge(1, 2, string='AATT')
    nx_graph.add_edge(3, 1, string='GGAA')
    nx_graph.add_edge(1, 4, string='AARR')

lightdb = LightMappedDBGraph.fromDB(DBLoop3(), string_set={},
                                    raw_mappings={0: ([(0, 1, 0), (1, 1, 0), (1, 2, 0)], 0, 0),
                                                  1: ([(3, 1, 0), (1, 1, 1), (1, 1, 1), (1, 4, 0)], 0, 0),
                                                  2: ([(3, 1, 0), (1, 2, 0)], 0, 0)})
lightdb.increase_k_by_one()
assert [(edge, lightdb.edge2index[edge]) for edge in lightdb.nx_graph.edges] == \
    [((0, 11, 0), 0), ((3, 8, 0), 5), ((7, 10, 0), 6), ((7, 4, 0), 4), ((8, 10, 0), 7), ((8, 11, 0), 8),
     ((10, 7, 0), 2), ((11, 2, 0), 3)]
lightdb.edge2seq
assert lightdb.edge2seq == \
    {0: ['C', 'C', 'A', 'A', 'C', 'A', 'A', 'T'],
     2: ['A', 'A', 'G', 'A', 'A'],
     3: ['A', 'A', 'T', 'T'],
     4: ['G', 'A', 'A', 'R', 'R'],
     5: ['G', 'G', 'A', 'A'],
     6: ['G', 'A', 'A', 'G'],
     7: ['G', 'A', 'A', 'G'],
     8: ['G', 'A', 'A', 'T']}

merge idb 0 1 (0, 5, 0) (9, 6, 0)
4 3
1 -> 0
[]


In [2337]:
# graph with a loop

class DBLoop2:
    k=3
    nx_graph = nx.MultiDiGraph()
    nx_graph.add_edge(0, 1, string='CCAA')
    nx_graph.add_edge(1, 1, string='AACAA')
    nx_graph.add_edge(1, 1, string='AAGAA')
    nx_graph.add_edge(1, 2, string='AATT')

lightdb = LightMappedDBGraph.fromDB(DBLoop2(), string_set={},
                                    raw_mappings={0: ([(0, 1, 0), (1, 1, 0), (1, 2, 0)], 0, 0),
                                                  1: ([(0, 1, 0), (1, 1, 1), (1, 2, 0)], 0, 0)})
lightdb.increase_k_by_one()
assert [(edge, lightdb.edge2index[edge]) for edge in lightdb.nx_graph.edges] == \
    [((0, 3, 0), 0), ((3, 8, 0), 1), ((3, 8, 1), 2), ((8, 2, 0), 3)]
lightdb.edge2seq
assert lightdb.edge2seq == \
    {0: ['C', 'C', 'A', 'A'],
     1: ['C', 'A', 'A', 'C', 'A', 'A', 'T'],
     2: ['C', 'A', 'A', 'G', 'A', 'A', 'T'],
     3: ['A', 'A', 'T', 'T']}

[]


In [2338]:
# graph with a loop

class DBLoop:
    k=3
    nx_graph = nx.MultiDiGraph()
    nx_graph.add_edge(0, 1, string='CCAA')
    nx_graph.add_edge(1, 1, string='AACAA')
    nx_graph.add_edge(1, 2, string='AATT')

lightdb = LightMappedDBGraph.fromDB(DBLoop(), string_set={},
                                    raw_mappings={0: ([(0, 1, 0), (1, 1, 0)], 0, 0),
                                                  1: ([(1, 1, 0), (1, 2, 0)], 0, 0)})
lightdb.increase_k_by_one()
assert [(edge, lightdb.edge2index[edge]) for edge in lightdb.nx_graph.edges] == \
    [((0, 2, 0), 0)]
lightdb.edge2seq
assert lightdb.edge2seq == \
    {0: ['C', 'C', 'A', 'A', 'C', 'A', 'A', 'T', 'T']}

merge idb 0 1 (0, 3, 0) (5, 4, 0)
4 3
1 -> 0
merge idb 0 2 (0, 4, 0) (6, 2, 0)
7 2
[]


In [2339]:
# graph with a complex vertex

class DBComplexVertex1:
    k=3
    nx_graph = nx.MultiDiGraph()
    nx_graph.add_edge(0, 2, string='ACAAA')
    nx_graph.add_edge(1, 2, string='GGAAA')
    nx_graph.add_edge(2, 3, string='AATGC')
    nx_graph.add_edge(2, 4, string='AATT')

lightdb = LightMappedDBGraph.fromDB(DBComplexVertex1(), string_set={},
                                    raw_mappings={0: ([(0, 2, 0), (2, 3, 0)], 0, 0),
                                                  1: ([(0, 2, 0), (2, 4, 0)], 0, 0),
                                                  2: ([(1, 2, 0), (2, 4, 0)], 0, 0)})
assert [(edge, lightdb.edge2index[edge]) for edge in lightdb.nx_graph.edges] == \
    [((0, 2, 0), 0), ((2, 3, 0), 1), ((2, 4, 0), 2), ((1, 2, 0), 3)]
lightdb.increase_k_by_one()
print([(edge, lightdb.edge2index[edge]) for edge in lightdb.nx_graph.edges])
print(lightdb.edge2seq)
assert [(edge, lightdb.edge2index[edge]) for edge in lightdb.nx_graph.edges] == \
    [((0, 5, 0), 0), ((1, 8, 0), 3), ((5, 3, 0), 1), ((5, 8, 0), 4), ((8, 4, 0), 2)]
assert lightdb.edge2seq == \
    {0: ['A', 'C', 'A', 'A', 'A'],
     1: ['A', 'A', 'A', 'T', 'G', 'C'],
     2: ['A', 'A', 'T', 'T'],
     3: ['G', 'G', 'A', 'A', 'A', 'T'],
     4: ['A', 'A', 'A', 'T']}

[]
[((0, 5, 0), 0), ((1, 8, 0), 3), ((5, 3, 0), 1), ((5, 8, 0), 4), ((8, 4, 0), 2)]
{0: ['A', 'C', 'A', 'A', 'A'], 1: ['A', 'A', 'A', 'T', 'G', 'C'], 2: ['A', 'A', 'T', 'T'], 3: ['G', 'G', 'A', 'A', 'A', 'T'], 4: ['A', 'A', 'A', 'T']}


In [2340]:
# graph with a complex vertex

class DBComplexVertex3:
    k=3
    nx_graph = nx.MultiDiGraph()
    nx_graph.add_edge(0, 2, string='CCAC')
    nx_graph.add_edge(1, 2, string='TTAC')
    nx_graph.add_edge(2, 5, string='ACG')
    nx_graph.add_edge(3, 5, string='AACG')
    nx_graph.add_edge(5, 4, string='CGTA')
    nx_graph.add_edge(5, 6, string='CGA')
    nx_graph.add_edge(6, 7, string='GACC')
    nx_graph.add_edge(6, 8, string='GATT')
    

lightdb = LightMappedDBGraph.fromDB(DBComplexVertex3(), string_set={},
                                    raw_mappings={0: ([(0, 2, 0), (2, 5, 0), (5, 6, 0), (6, 7, 0)], 0, 0),
                                                  1: ([(1, 2, 0), (2, 5, 0), (5, 4, 0)], 0, 0),
                                                  2: ([(3, 5, 0), (5, 6, 0), (6, 8, 0)], 0, 0)})
assert [(edge, lightdb.edge2index[edge]) for edge in lightdb.nx_graph.edges] == \
    [((0, 2, 0), 0), ((2, 5, 0), 1), ((5, 4, 0), 3), ((5, 6, 0), 4), ((1, 2, 0), 2),
     ((6, 7, 0), 6), ((6, 8, 0), 7), ((3, 5, 0), 5)]
lightdb.increase_k_by_one()
assert [(edge, lightdb.edge2index[edge]) for edge in lightdb.nx_graph.edges] == \
    [((0, 2, 0), 0), ((2, 4, 0), 3), ((2, 12, 0), 8), ((1, 2, 0), 2), ((3, 12, 0), 5),
     ((12, 7, 0), 6), ((12, 8, 0), 7)]
assert lightdb.edge2seq == \
    {0: ['C', 'C', 'A', 'C', 'G'],
     2: ['T', 'T', 'A', 'C', 'G'],
     3: ['A', 'C', 'G', 'T', 'A'],
     5: ['A', 'A', 'C', 'G', 'A'],
     6: ['C', 'G', 'A', 'C', 'C'],
     7: ['C', 'G', 'A', 'T', 'T'],
     8: ['A', 'C', 'G', 'A']}

[(1, (2, 9, 0)), (4, (12, 6, 0))]


In [2341]:
# graph with a complex vertex

class DBComplexVertex2:
    k=3
    nx_graph = nx.MultiDiGraph()
    nx_graph.add_edge(0, 2, string='ACAAA')
    nx_graph.add_edge(1, 2, string='GGAAA')
    nx_graph.add_edge(2, 3, string='AATGC')
    nx_graph.add_edge(2, 4, string='AATT')

lightdb = LightMappedDBGraph.fromDB(DBComplexVertex2(), string_set={},
                                    raw_mappings={0: ([(0, 2, 0), (2, 3, 0)], 0, 0),
                                                  1: ([(0, 2, 0), (2, 4, 0)], 0, 0),
                                                  2: ([(1, 2, 0), (2, 4, 0)], 0, 0),
                                                  3: ([(1, 2, 0), (2, 3, 0)], 0, 0)})
assert [(edge, lightdb.edge2index[edge]) for edge in lightdb.nx_graph.edges] == \
    [((0, 2, 0), 0), ((2, 3, 0), 1), ((2, 4, 0), 2), ((1, 2, 0), 3)]
lightdb.increase_k_by_one()
assert [(edge, lightdb.edge2index[edge]) for edge in lightdb.nx_graph.edges] == \
    [((0, 5, 0), 0), ((1, 6, 0), 3), ((5, 7, 0), 4), ((5, 8, 0), 5), ((6, 7, 0), 6),
     ((6, 8, 0), 7), ((7, 3, 0), 1), ((8, 4, 0), 2)]
assert lightdb.edge2seq == \
    {0: ['A', 'C', 'A', 'A', 'A'],
     1: list('AATGC'),
     2: list('AATT'),
     3: ['G', 'G', 'A', 'A', 'A'],
     4: list('AAAT'),
     5: list('AAAT'),
     6: list('AAAT'),
     7: list('AAAT')}

[]


In [2342]:
# graph starting vertex

class DBStVertex:
    k=3
    nx_graph = nx.MultiDiGraph()
    nx_graph.add_edge(0, 1, string='AAAAA')
    nx_graph.add_edge(0, 2, string='AAACA')
    nx_graph.add_edge(0, 3, string='AAA')    

    
lightdb = LightMappedDBGraph.fromDB(DBStVertex(), string_set={}, raw_mappings={0: ([(0, 1, 0)], 0, 0),
                                                                               1: ([(0, 2, 0)], 0, 0),
                                                                               2: ([(0, 3, 0)], 0, 0)})
assert lightdb.idb_mappings.mappings == {0: [0], 1: [1], 2: [2]}
lightdb.increase_k_by_one()
assert [(edge, lightdb.edge2index[edge]) for edge in lightdb.nx_graph.edges] == \
        [((0, 1, 0), 0), ((4, 2, 0), 1)]
assert lightdb.idb_mappings.mappings == {0: [0], 1: [1], 2: []}

[(2, (5, 3, 0))]


In [2343]:
# graph ending vertex

class DBEnVertex:
    k=3
    nx_graph = nx.MultiDiGraph()
    nx_graph.add_edge(0, 3, string='AAAA')
    nx_graph.add_edge(1, 3, string='AACA')
    nx_graph.add_edge(2, 3, string='AAA')
    


lightdb = LightMappedDBGraph.fromDB(DBEnVertex(), string_set={}, raw_mappings={0: ([(0, 3, 0)], 0, 0),
                                                                               1: ([(1, 3, 0)], 0, 0),
                                                                               2: ([(2, 3, 0)], 0, 0)})
assert lightdb.idb_mappings.mappings == {0: [0], 1: [1], 2: [2]}
lightdb.increase_k_by_one()
assert [(edge, lightdb.edge2index[edge]) for edge in lightdb.nx_graph.edges] == \
        [((0, 3, 0), 0), ((1, 4, 0), 1)]
assert lightdb.idb_mappings.mappings == {0: [0], 1: [1], 2: []}

[(2, (2, 5, 0))]


In [2344]:
# graph 1-in >1-out

class DB1inVertex:
    k=3
    nx_graph = nx.MultiDiGraph()
    nx_graph.add_edge(0, 1, string='AACAG')
    nx_graph.add_edge(1, 2, string='AGACC')
    nx_graph.add_edge(1, 3, string='AGATT')
    nx_graph.add_edge(1, 4, string='AGAGG')
    

lightdb = LightMappedDBGraph.fromDB(DB1inVertex(), string_set={}, raw_mappings={})
assert [lightdb.edge2seq[lightdb.edge2index[edge]] for edge in lightdb.nx_graph.edges] == \
    [list('AACAG'), ['A', 'G', 'A', 'C', 'C'], ['A', 'G', 'A', 'T', 'T'], ['A', 'G', 'A', 'G', 'G']]
lightdb.increase_k_by_one()
assert [lightdb.edge2seq[lightdb.edge2index[edge]] for edge in lightdb.nx_graph.edges] == \
    [list('AACAG'), ['C', 'A', 'G', 'A', 'C', 'C'], ['C', 'A', 'G', 'A', 'T', 'T'], ['C', 'A', 'G', 'A', 'G', 'G']]

[]


In [2345]:
# graph >1-in 1-out

class DB1outVertex:
    k=3
    nx_graph = nx.MultiDiGraph()
    nx_graph.add_edge(0, 3, string='CCAGA')
    nx_graph.add_edge(1, 3, string='TTAGA')
    nx_graph.add_edge(2, 3, string='GGAGA')
    nx_graph.add_edge(3, 4, string='GAAAA')
    

lightdb = LightMappedDBGraph.fromDB(DB1outVertex(), string_set={}, raw_mappings={})
assert sorted([lightdb.edge2seq[lightdb.edge2index[edge]] for edge in lightdb.nx_graph.edges]) == \
    sorted([list('CCAGA'), list('TTAGA'), list('GGAGA'), list('GAAAA')])
lightdb.increase_k_by_one()
assert sorted([lightdb.edge2seq[lightdb.edge2index[edge]] for edge in lightdb.nx_graph.edges]) == \
    sorted([list('CCAGAA'), list('TTAGAA'), list('GGAGAA'), list('GAAAA')])

[]


In [2373]:
# real graph

lightdb = LightMappedDBGraph.fromDB(dbs_assembly[k], string_set=monoreads_set, neutral_symbs=set('?'),
                                    raw_mappings=mappings)

In [2374]:
K = 2000

In [2375]:
for i in range(K-k):
    print(i, lightdb.increase_k_by_one())

[(2, (102, 3793, 0)), (6, (104, 105, 0))]
0 None
[(10, (106, 3798, 0)), (3, (3795, 3810, 0))]
1 None
[(11, (3803, 3790, 0)), (25, (3806, 3821, 0))]
2 None
[(0, (3791, 3804, 0)), (7, (3811, 3815, 0))]
3 None
[(26, (3816, 3826, 0)), (28, (3824, 3841, 0))]
4 None
[(30, (3833, 3851, 0)), (27, (3839, 3829, 0))]
5 None
[(24, (3827, 3834, 0)), (29, (3842, 3847, 0))]
6 None
[(34, (3854, 3874, 0))]
7 None
[(31, (3848, 3868, 0)), (36, (3858, 3878, 0)), (32, (3867, 3857, 0)), (33, (3872, 3862, 0))]
8 None
[]
9 None
[(35, (3875, 3882, 0)), (39, (3883, 3897, 0))]
10 None
[(40, (3880, 3903, 0)), (37, (3895, 3886, 0)), (38, (3900, 3890, 0))]
11 None
[(41, (3888, 3913, 0)), (42, (3905, 3908, 0))]
12 None
[(45, (3911, 3922, 0)), (46, (3914, 3934, 0))]
13 None
[(44, (3927, 3919, 0)), (48, (3931, 3945, 0))]
14 None
[(43, (3920, 3928, 0)), (47, (3936, 3939, 0))]
15 None
[(50, (3942, 3950, 0)), (52, (3948, 3965, 0))]
16 None
[(54, (3957, 3975, 0)), (51, (3963, 3953, 0))]
17 None
[(49, (3952, 3958, 0)), (53

[(309, (5240, 5263, 0)), (306, (5256, 5248, 0)), (307, (5260, 5252, 0))]
143 None
[(310, (5250, 5273, 0)), (311, (5264, 5268, 0))]
144 None
[(314, (5271, 5282, 0)), (315, (5276, 5293, 0))]
145 None
[(313, (5286, 5279, 0)), (317, (5289, 5303, 0))]
146 None
[(312, (5281, 5288, 0)), (316, (5295, 5298, 0))]
147 None
[(319, (5299, 5308, 0)), (321, (5306, 5323, 0))]
148 None
[(323, (5315, 5332, 0)), (320, (5321, 5312, 0))]
149 None
[(318, (5309, 5317, 0)), (322, (5326, 5328, 0))]
150 None
[(327, (5333, 5353, 0))]
151 None
[(324, (5330, 5347, 0)), (329, (5340, 5357, 0)), (325, (5346, 5337, 0)), (326, (5351, 5342, 0))]
152 None
[]
153 None
[(328, (5354, 5361, 0)), (332, (5363, 5375, 0))]
154 None
[(333, (5358, 5381, 0)), (330, (5374, 5366, 0)), (331, (5378, 5370, 0))]
155 None
[(334, (5368, 5391, 0)), (335, (5382, 5386, 0))]
156 None
[(338, (5389, 5400, 0)), (339, (5394, 5411, 0))]
157 None
[(337, (5404, 5397, 0)), (341, (5407, 5421, 0))]
158 None
[(336, (5399, 5406, 0)), (340, (5413, 5416, 0)

293 None
[(580, (6556, 6564, 0)), (584, (6570, 6573, 0))]
294 None
[]
295 None
[(585, (6575, 6587, 0)), (588, (6578, 6592, 0)), (586, (6586, 6577, 0)), (587, (6590, 6583, 0))]
296 None
[]
297 None
[(589, (6594, 6596, 0)), (592, (6597, 6609, 0))]
298 None
[(590, (6608, 6600, 0)), (591, (6613, 6605, 0))]
299 None
[(593, (6602, 6615, 0)), (594, (6616, 6619, 0))]
300 None
[(597, (6622, 6629, 0))]
301 None
[(596, (6632, 6625, 0)), (598, (6635, 6638, 0))]
302 None
[(595, (6627, 6634, 0)), (599, (6641, 6643, 0))]
303 None
[(601, (6644, 6648, 0))]
304 None
[(603, (6655, 6662, 0)), (602, (6660, 6652, 0))]
305 None
[(600, (6650, 6658, 0)), (604, (6663, 6667, 0))]
306 None
[]
307 None
[(605, (6669, 6681, 0)), (608, (6672, 6686, 0)), (606, (6680, 6671, 0)), (607, (6684, 6677, 0))]
308 None
[]
309 None
[(609, (6688, 6690, 0)), (612, (6691, 6703, 0))]
310 None
[(610, (6702, 6694, 0)), (611, (6707, 6699, 0))]
311 None
[(613, (6696, 6709, 0)), (614, (6710, 6713, 0))]
312 None
[(617, (6716, 6723, 0))]


468 None
[(873, (7904, 7911, 0))]
469 None
[(872, (7914, 7907, 0))]
470 None
[(870, (7909, 7898, 0)), (875, (7918, 7920, 0))]
471 None
[(877, (7921, 7925, 0))]
472 None
[(874, (7931, 7916, 0))]
473 None
[(876, (7927, 7930, 0)), (879, (7936, 7939, 0))]
474 None
[]
475 None
[(880, (7941, 7947, 0)), (878, (7946, 7934, 0)), (881, (7950, 7944, 0))]
476 None
[]
477 None
[(883, (7954, 7956, 0)), (885, (7957, 7965, 0))]
478 None
[(882, (7964, 7952, 0)), (884, (7968, 7961, 0))]
479 None
[(887, (7971, 7974, 0))]
480 None
[(889, (7976, 7983, 0))]
481 None
[(888, (7986, 7979, 0))]
482 None
[(886, (7981, 7970, 0)), (891, (7990, 7992, 0))]
483 None
[(893, (7993, 7997, 0))]
484 None
[(890, (8003, 7988, 0))]
485 None
[(892, (7999, 8002, 0)), (895, (8008, 8011, 0))]
486 None
[]
487 None
[(896, (8013, 8019, 0)), (894, (8018, 8006, 0)), (897, (8022, 8016, 0))]
488 None
[]
489 None
[(899, (8026, 8028, 0)), (901, (8029, 8037, 0))]
490 None
[(898, (8036, 8024, 0)), (900, (8040, 8033, 0))]
491 None
[(903, (8

[(1126, (9065, 9054, 0)), (1131, (9074, 9076, 0))]
663 None
[(1133, (9077, 9081, 0))]
664 None
[(1130, (9087, 9072, 0))]
665 None
[(1132, (9083, 9086, 0)), (1135, (9092, 9095, 0))]
666 None
[]
667 None
[(1136, (9097, 9103, 0)), (1134, (9102, 9090, 0)), (1137, (9106, 9100, 0))]
668 None
[]
669 None
[(1139, (9110, 9112, 0)), (1141, (9113, 9121, 0))]
670 None
[(1138, (9120, 9108, 0)), (1140, (9124, 9117, 0))]
671 None
[(1143, (9127, 9130, 0))]
672 None
[(1145, (9132, 9139, 0))]
673 None
[(1144, (9142, 9135, 0))]
674 None
[(1142, (9137, 9126, 0)), (1147, (9146, 9148, 0))]
675 None
[(1149, (9149, 9153, 0))]
676 None
[(1146, (9159, 9144, 0))]
677 None
[(1148, (9155, 9158, 0)), (1151, (9164, 9167, 0))]
678 None
[]
679 None
[(1152, (9169, 9175, 0)), (1150, (9174, 9162, 0)), (1153, (9178, 9172, 0))]
680 None
[]
681 None
[(1155, (9182, 9184, 0)), (1157, (9185, 9193, 0))]
682 None
[(1154, (9192, 9180, 0)), (1156, (9196, 9189, 0))]
683 None
[(1159, (9199, 9202, 0))]
684 None
[(1161, (9204, 9211, 0

[(1394, (10272, 10260, 0)), (1396, (10276, 10269, 0))]
863 None
[(1399, (10279, 10282, 0))]
864 None
[(1401, (10284, 10291, 0))]
865 None
[(1400, (10294, 10287, 0))]
866 None
[(1398, (10289, 10278, 0)), (1403, (10298, 10300, 0))]
867 None
[(1405, (10301, 10305, 0))]
868 None
[(1402, (10311, 10296, 0))]
869 None
[(1404, (10307, 10310, 0)), (1407, (10316, 10319, 0))]
870 None
[]
871 None
[(1408, (10321, 10327, 0)), (1406, (10326, 10314, 0)), (1409, (10330, 10324, 0))]
872 None
[]
873 None
[(1411, (10334, 10336, 0)), (1413, (10337, 10345, 0))]
874 None
[(1410, (10344, 10332, 0)), (1412, (10348, 10341, 0))]
875 None
[(1415, (10351, 10354, 0))]
876 None
[(1417, (10356, 10363, 0))]
877 None
[(1416, (10366, 10359, 0))]
878 None
[(1414, (10361, 10350, 0)), (1419, (10370, 10372, 0))]
879 None
[(1421, (10373, 10377, 0))]
880 None
[(1418, (10383, 10368, 0))]
881 None
[(1420, (10379, 10382, 0)), (1423, (10388, 10391, 0))]
882 None
[]
883 None
[(1424, (10393, 10399, 0)), (1422, (10398, 10386, 0)), 

[(1644, (11378, 11364, 0))]
1049 None
[(1646, (11374, 11377, 0)), (1649, (11384, 11386, 0))]
1050 None
[]
1051 None
[(1650, (11388, 11393, 0)), (1648, (11392, 11381, 0)), (1651, (11396, 11390, 0))]
1052 None
[]
1053 None
[(1653, (11399, 11402, 0)), (1655, (11403, 11410, 0))]
1054 None
[(1652, (11409, 11398, 0)), (1654, (11413, 11406, 0))]
1055 None
[(1657, (11416, 11419, 0))]
1056 None
[(1659, (11421, 11427, 0))]
1057 None
[(1658, (11430, 11423, 0))]
1058 None
[(1656, (11425, 11415, 0)), (1661, (11434, 11436, 0))]
1059 None
[(1663, (11437, 11441, 0))]
1060 None
[(1660, (11446, 11432, 0))]
1061 None
[(1662, (11442, 11445, 0)), (1665, (11452, 11454, 0))]
1062 None
[]
1063 None
[(1666, (11456, 11461, 0)), (1664, (11460, 11449, 0)), (1667, (11464, 11458, 0))]
1064 None
[]
1065 None
[(1669, (11467, 11470, 0)), (1671, (11471, 11478, 0))]
1066 None
[(1668, (11477, 11466, 0)), (1670, (11481, 11474, 0))]
1067 None
[(1673, (11484, 11487, 0))]
1068 None
[(1675, (11489, 11495, 0))]
1069 None
[(167

[(1905, (12481, 12492, 0)), (1903, (12490, 12479, 0))]
1287 None
[]
1288 None
[]
1289 None
[(1908, (12499, 12506, 0)), (1907, (12504, 12497, 0))]
1290 None
[]
1291 None
[(1906, (12508, 12501, 0))]
1292 None
[]
1293 None
[(1909, (12516, 12510, 0))]
1294 None
[(1911, (12511, 12518, 0)), (1910, (12521, 12514, 0))]
1295 None
[]
1296 None
[]
1297 None
[(1914, (12528, 12532, 0))]
1298 None
[(1912, (12525, 12526, 0)), (1913, (12534, 12523, 0))]
1299 None
[]
1300 None
[]
1301 None
[(1917, (12537, 12545, 0)), (1916, (12543, 12535, 0))]
1302 None
[]
1303 None
[(1915, (12547, 12540, 0))]
1304 None
[]
1305 None
[(1918, (12555, 12549, 0))]
1306 None
[(1920, (12550, 12557, 0)), (1919, (12560, 12553, 0))]
1307 None
[]
1308 None
[]
1309 None
[(1923, (12569, 12571, 0))]
1310 None
[(1921, (12563, 12565, 0)), (1922, (12573, 12562, 0))]
1311 None
[]
1312 None
[]
1313 None
[(1926, (12576, 12584, 0)), (1925, (12582, 12574, 0))]
1314 None
[]
1315 None
[(1924, (12586, 12579, 0))]
1316 None
[]
1317 None
[(1927

1602 None
[]
1603 None
[]
1604 None
[]
1605 None
[(2066, (13161, 13163, 0))]
1606 None
[]
1607 None
[]
1608 None
[]
1609 None
[]
1610 None
[]
1611 None
[]
1612 None
[]
1613 None
[(2067, (13165, 13168, 0))]
1614 None
[]
1615 None
[]
1616 None
[]
1617 None
[(2068, (13169, 13171, 0))]
1618 None
[]
1619 None
[]
1620 None
[]
1621 None
[]
1622 None
[]
1623 None
[]
1624 None
[]
1625 None
[(2069, (13173, 13176, 0))]
1626 None
[]
1627 None
[]
1628 None
[]
1629 None
[(2070, (13177, 13179, 0))]
1630 None
[]
1631 None
[]
1632 None
[(14, (12716, 2728, 0))]
1633 None
[]
1634 None
[]
1635 None
[]
1636 None
[]
1637 None
[(2071, (13182, 13184, 0))]
1638 None
[]
1639 None
[]
1640 None
[]
1641 None
[(2072, (13185, 13187, 0))]
1642 None
[]
1643 None
[]
1644 None
[]
1645 None
[]
1646 None
[]
1647 None
[]
1648 None
[]
1649 None
[(2073, (13189, 13192, 0))]
1650 None
[]
1651 None
[]
1652 None
[]
1653 None
[(2074, (13193, 13195, 0))]
1654 None
[]
1655 None
[]
1656 None
[]
1657 None
[]
1658 None
[]
1659 None
[]

In [2376]:
transformed_db = lightdb.toDB()

In [2377]:
transformed_db.write_dot(f'../../experiments/{date}/transformed_test', compact=True)

# Testing correctness

In [2378]:
dbs_assembly_reference, _ = \
    get_idb_monostring_set(string_set=sd_report_assembly.monostring_set,
                           mink=K, maxk=K,
                           outdir=None,
                           mode='assembly')

In [2379]:
color3graph = DeBruijnGraph3Color.from_db_graphs(gr_assembly=dbs_assembly_reference[K],
                                                 gr_reads=transformed_db)

In [2380]:
color3graph.write_dot(f'../../experiments/{date}/c3g_test', compact=True)

In [2381]:
!realpath ../../experiments/{date}/c3g_test

/Poppy/abzikadze/centroFlye/centroFlye_repo/experiments/20200928/c3g_test
